## 5 minute bar - chart pattern - ananylics

In [1]:
import warnings
warnings.filterwarnings('ignore')

import time
from datetime import date, datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta

import matplotlib.pyplot as plt
import pandas as pd
import pandas_ta as ta
import numpy as np

In [2]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

## Date 2024-07-06 Analytics

## Date 2024-07-06 Analytics

In [3]:
data = dataset.copy()
data['lower_shadow'] = data.apply(
        lambda r: min(r['Open'], r['Close']) - r['Low'], axis=1)
data['upper_shadow'] = data.apply(
        lambda r: r['High'] - max(r['Open'], r['Close']), axis=1)
data['body'] = data.apply(
        lambda r: abs(r['Close'] - r['Open']), axis=1)
data["ema_f"] = ta.ema(data["Close"], length=5)
data["ema_l"] = ta.ema(data["Close"], length=26)
data["entry_price"] = data["Open"].shift(-1)
_1_d_df = data.copy()
_1_d_df = _1_d_df[100 * _1_d_df.index.hour + _1_d_df.index.minute < 1430]
_1_d_df = data[['Close']].resample("D").agg({
        'Close': 'last'
    }).rename(columns={'Close': 'exit_price'})
_1_d_df.dropna(inplace=True)
data = data.assign(time_d=pd.PeriodIndex(data.index, freq='1D').to_timestamp())
data = pd.merge(data, _1_d_df, left_on="time_d", right_index=True, how="left")
data['long_return'] = data.apply(
        lambda r: r['exit_price'] - r['entry_price'], axis=1)
data['short_return'] = data.apply(
        lambda r: r['entry_price'] - r['exit_price'], axis=1)
data['color'] = data.apply(
        lambda r: 'doji' if r['Open'] == r['Close'] else ('xanh' if r['Open'] < r['Close'] else 'do' ), axis=1)

In [4]:
data

,Open,High,Low,Close,Volume,lower_shadow,upper_shadow,body,ema_f,ema_l,entry_price,time_d,exit_price,long_return,short_return,color
Date,,,,,,,,,,,,,,,,
2018-08-13 09:00:00,943.5,943.6,942.9,943.1,1812,0.2,0.1,0.4,NaN,NaN,943.1,2018-08-13,954.2,11.1,-11.1,do
2018-08-13 09:05:00,943.1,943.5,942.9,943.3,1323,0.2,0.2,0.2,NaN,NaN,943.2,2018-08-13,954.2,11.0,-11.0,xanh
2018-08-13 09:10:00,943.2,943.3,942.6,943.1,1207,0.5,0.1,0.1,NaN,NaN,943.1,2018-08-13,954.2,11.1,-11.1,do
2018-08-13 09:15:00,943.1,943.1,942.3,942.6,1196,0.3,0.0,0.5,NaN,NaN,942.6,2018-08-13,954.2,11.6,-11.6,do
2018-08-13 09:20:00,942.6,943.7,942.4,943.7,1765,0.2,0.0,1.1,943.160000,NaN,943.8,2018-08-13,954.2,10.4,-10.4,xanh
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-09 14:15:00,1321.3,1321.9,1318.4,1319.1,11670,0.7,0.6,2.2,1318.355191,1315.478801,1319.0,2024-07-09,1320.5,1.5,-1.5,do
2024-07-09 14:20:00,1319.0,1319.0,1317.3,1317.5,6217,0.2,0.0,1.5,1318.070127,1315.628519,1317.5,2024-07-09,1320.5,3.0,-3.0,do
2024-07-09 14:25:00,1317.5,1319.9,1317.5,1319.5,6566,0.0,0.4,2.0,1318.546752,1315.915295,1319.4,2024-07-09,1320.5,1.1,-1.1,xanh


In [ ]:
# Only trade from 1:45PM to 2:25PM
good_time_data = data[(100*data.index.hour + data.index.minute > 1340) & (100*data.index.hour + data.index.minute < 1430)]

In [ ]:
good_time_data

In [ ]:
long_lower_shadow = good_time_data[good_time_data.lower_shadow > 1.6]
long_lower_shadow

In [ ]:
long_lower_shadow['rate_1'] = long_lower_shadow.apply(
        lambda r: r['upper_shadow'] / r['lower_shadow'], axis=1)
long_lower_shadow['rate_2'] = long_lower_shadow.apply(
        lambda r: r['body'] / r['lower_shadow'], axis=1)
long_lower_shadow['rate_3'] = long_lower_shadow.apply(
        lambda r: (r['body'] + r['upper_shadow']) / r['lower_shadow'], axis=1)
long_lower_shadow['rate_3'] = long_lower_shadow['rate_3'].round(2)

In [ ]:
y = long_lower_shadow[(long_lower_shadow.ema_f < long_lower_shadow.ema_l) & (long_lower_shadow.color == 'xanh') & (long_lower_shadow.rate_3 < 0.35)]
total_profit = y['long_return'].sum()
total_profit

In [ ]:
long_lower_shadow_should_buy = long_lower_shadow[long_lower_shadow['long_return'] > 0]
long_lower_shadow_should_not_buy = long_lower_shadow[long_lower_shadow['long_return'] < 0]

In [ ]:
long_lower_shadow_should_buy[["lower_shadow", "upper_shadow", "body", "rate_1", "rate_2", "rate_3", "long_return"]]

In [ ]:
long_lower_shadow_should_buy[long_lower_shadow_should_buy.color == 'xanh']["rate_3"].describe()

In [ ]:
long_lower_shadow_should_buy[long_lower_shadow_should_buy.color == 'do']["rate_3"].describe()

In [ ]:
long_lower_shadow_should_buy

In [ ]:
long_lower_shadow_should_buy[["long_return"]].describe()

In [ ]:
long_lower_shadow_should_buy2 = long_lower_shadow[(long_lower_shadow['long_return'] > 3) & (long_lower_shadow['long_return'] < 11)]
# long_lower_shadow_should_buy2[["rate_3"]].describe()
long_lower_shadow_should_buy2[["rate_2"]].describe()

In [ ]:
a = long_lower_shadow_should_buy[long_lower_shadow_should_buy.ema_f < long_lower_shadow_should_buy.ema_l]
len(a)

In [ ]:
a[["rate_2"]].describe()

In [ ]:
b = long_lower_shadow_should_buy[long_lower_shadow_should_buy.ema_f > long_lower_shadow_should_buy.ema_l]
len(b)

In [ ]:
c = long_lower_shadow_should_not_buy[long_lower_shadow_should_not_buy.ema_f < long_lower_shadow_should_not_buy.ema_l]
len(c)

In [ ]:
plt.plot(long_lower_shadow_should_buy['rate_1'], long_lower_shadow_should_buy['rate_2'], 'o')

In [ ]:
plt.plot(long_lower_shadow_should_not_buy['rate_1'], long_lower_shadow_should_not_buy['rate_2'], 'o')

In [ ]:
data02 = dataset.copy()
data02['h1'] = data02['High'] - data02['High'].shift(1)
data02['h2'] = data02['High'] - data02['High'].shift(2)
data02['c1'] = data02['Close'] - data02['Close'].shift(1)
data02['c2'] = data02['Close'] - data02['Close'].shift(2)
data02['o1'] = data02['Open'] - data02['Open'].shift(1)
data02['o2'] = data02['Open'] - data02['Open'].shift(2)
data02['l1'] = data02['Low'] - data02['Low'].shift(1)
data02['l2'] = data02['Low'] - data02['Low'].shift(2)
data02.dropna(inplace=True)
k = data02[['h1', 'h2', 'c1', 'c2', 'o1', 'o2', 'l1', 'l2']]
k

In [ ]:
# import pickle
# trainner_file = str(algo_dir) + '/VN30ps/kmean_clusters/kmeans.pickle'

In [ ]:
from sklearn.cluster import KMeans
X = k.to_numpy()
trainner_file_existed = os.path.isfile(trainner_file)
# if trainner_file_existed:
#     with open(trainner_file, 'rb') as f:
#         kmeans = pickle.load(f)
# else:
#     kmeans = KMeans(n_clusters=39, random_state=0).fit(X)
#     with open(trainner_file, 'wb') as fp:
#         pickle.dump(kmeans, fp)
kmeans = KMeans(n_clusters=39, random_state=0).fit(X)
pred_label = kmeans.predict(X)
pred_label

In [ ]:
k = k.assign(Nhom=pred_label)

In [ ]:
k

In [ ]:
k[k.index == '2024-06-19 10:05:00']

In [ ]:
k[k.index == '2024-06-19 10:10:00']

In [ ]:
k[k.index == '2024-06-19 10:20:00']

In [ ]:
k[(k.index == '2024-06-19 10:05:00') | (k.index == '2024-06-19 13:45:00') | (k.index == '2024-06-19 13:50:00')]

In [ ]:
k[k.Nhom == 7]

In [ ]:
k[((k.index > '2024-06-18 09:15:00') & (k.index < '2024-06-18 14:15:00') & (k.Nhom == 0)) | (k.index == '2024-06-19 10:20:00')]

In [ ]:
k[k.index == '2024-06-19 14:20:00']

In [ ]:
k[((k.index > '2024-06-18 09:15:00') & (k.index < '2024-06-18 14:15:00') & (k.Nhom == 2)) | (k.index == '2024-06-19 14:20:00')]

In [ ]:
k[k.index == '2024-06-18 14:00:00']

In [ ]:
data03 = dataset.copy()
#upper shadow
data03['us_1'] = data03.apply(
    lambda r: r['High'] - max(r['Open'], r['Close']), axis=1
)
data03['bd_1'] = data03.apply(
    lambda r: r['Close'] - r['Open'], axis=1
)
data03['ls_1'] = data03.apply(
    lambda r: min(r['Open'], r['Close']) - r['Low'], axis=1
)
data03['us_2'] = data03['us_1'].shift(1)
data03['bd_2'] = data03['bd_1'].shift(1)
data03['ls_2'] = data03['ls_1'].shift(1)
data03['us_3'] = data03['us_1'].shift(2)
data03['bd_3'] = data03['bd_1'].shift(2)
data03['ls_3'] = data03['ls_1'].shift(2)
data03.dropna(inplace=True)
k2 = data03[['us_1', 'bd_1', 'ls_1', 'us_2', 'bd_2', 'ls_2', 'us_3', 'bd_3', 'ls_3']]
k2

In [ ]:
X2 = k2.to_numpy()
kmeans = KMeans(n_clusters=19, random_state=0).fit(X2)
g2 = kmeans.predict(X2)
g2

In [ ]:
k2 = k2.assign(Nhom=g2)

In [ ]:
k2[(k2.index == '2024-06-18 14:00:00') | (k2.index == '2024-06-18 14:05:00')]

In [ ]:
k2[((k2.index > '2024-06-19 09:15:00') & (k2.index < '2024-06-19 14:15:00') & (k2.Nhom == 10)) | (k2.index == '2024-06-18 14:00:00')]

In [ ]:
k2[k2.Nhom == 10]